In [20]:
#Imports
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
from functools import reduce
import os


In [95]:
df = pd.DataFrame(columns={"direction","lane","channel","date","hour","incident"})
df = df[["direction","lane","channel","date","hour","incident"]]

In [21]:
# MOVING FILES
#def movefiles(files):
#    list1 = files
#    allequips(list1)
#    list2 = [s.strip() for s in list1]
    
#    for item in list2:
#        open('C:\DAI db/%s.txt'%(item,), 'w')

In [124]:
def allequips(files):
    for file in files:
        
        f_name, f_ext = os.path.splitext(os.path.basename(file))
        spl_incident, channel, lane, direction, spl_date, spl_hour, ended = f_name.split(sep="_", maxsplit=6)
        incident = spl_incident.replace("(Event)", "")
        date = datetime.strptime(spl_date, "%Y%m%d").date()
        hour = datetime.strptime(spl_hour, "%H%M%S%f").time().replace(microsecond=0)
        df2 = pd.DataFrame([[direction, lane, channel, date, hour, incident]], columns=['direction', 'lane', 'channel', 'date', 'hour', 'incident'])
        globals()['df'] = df.append(df2)

In [122]:
#GET PATH
ini_folder = 'C:\Camaras DAI'
equips = map(lambda x: os.path.join(ini_folder, x, 'Traffic Incident'), os.listdir(ini_folder))
for equip in equips:
    dates = map(lambda x: os.path.join(equip, x), os.listdir(equip))
    days = []
    [days.extend(x) for x in map(lambda x: list(map(lambda y: os.path.join(x, y), os.listdir(x))), dates)]
    files = []
    [files.extend(x) for x in map(lambda x: list(map(lambda y: os.path.join(x, y), os.listdir(x))), days)]
    
    allequips(files)


Driving on the Lane Line(Event)_1_01_Up_20210629_135352909_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_135352909_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_140209210_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_140224039_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_140224877_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_141025186_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_141402650_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_142325266_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_142509580_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_142629252_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_143928641_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_144304715_unknown_P1.jpg
Driving on the Lane Line(Event)_1_01_Up_20210629_144550111_unknown_P1.jpg
Driving on the Lane Line(Event)_1_02_U

In [105]:
user = "root"
passwd = "trcvbr18"
host = "localhost"
port = 3306
db = "tracevia_app"

#Create connection to the MySQL database
engine = create_engine(f'mysql://{user}:{passwd}@{host}:{port}/{db}')
con = engine.connect()

In [109]:
df
df.to_sql(con=con, schema='tracevia_app', name='dai_history', if_exists='append', index=False) 

,direction,lane,channel,date,hour,incident
0,Up,01,1,2021-06-29,13:53:52,Driving on the Lane Line
0,Up,01,1,2021-06-29,13:53:52,Driving on the Lane Line
0,Up,01,1,2021-06-29,14:02:09,Driving on the Lane Line
0,Up,01,1,2021-06-29,14:02:24,Driving on the Lane Line
0,Up,01,1,2021-06-29,14:02:24,Driving on the Lane Line
...,...,...,...,...,...,...
0,Up,01,3,2021-06-29,16:51:44,Illegal Parking
0,Up,01,3,2021-06-29,16:52:31,Illegal Parking
0,Up,01,3,2021-06-29,16:54:13,Illegal Parking
0,Up,01,3,2021-06-29,16:55:00,Illegal Parking
